In [ ]:
import pandas as pd
import numpy as np
import json
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import string
from nltk.tokenize import word_tokenize
import nltk
import scipy.sparse as sp
ds = pd.read_csv("../data/processed/wikipedia_m.csv")
df_big = pd.read_csv("../data/raw/wikipedia_science_chunked_small_rag_512.csv")
df_small = df_big.sample(n=32000, random_state=42122)
df_small

,text,category,url,title
300201,Sommer and Behles A 19th-century Italian photo...,Engineering&Technology,https://en.wikipedia.org/wiki?curid=8874959,Sommer and Behles
869203,Hague Convention for the Protection of Cultura...,Humanities,https://en.wikipedia.org/wiki?curid=2578344,Hague Convention for the Protection of Cultu...
523623,Email spam The amount of spam that users see i...,Social_sciences,https://en.wikipedia.org/wiki?curid=459847,Email spam
616638,Direct debit The payer can cancel the authoriz...,Business&Economics,https://en.wikipedia.org/wiki?curid=2600081,Direct debit
186274,"TCP-seq However, it captures positions of only...",Biology,https://en.wikipedia.org/wiki?curid=51165385,TCP-seq
...,...,...,...,...
1106115,"Dambana Most of the remaining ""dambanas"" are n...",Religion&Philosophy&Ethics,https://en.wikipedia.org/wiki?curid=26211048,Dambana
493717,"Memory Several genes, proteins and enzymes hav...",Social_sciences,https://en.wikipedia.org/wiki?curid=31217535,Memory
437418,Tilt-A-Whirl is a flat ride similar to the Wal...,Mechanical_engineering,https://en.wikipedia.org/wiki?curid=1612404,Tilt-A-Whirl
779043,Judicial system in the United Arab Emirates Th...,Humanities,https://en.wikipedia.org/wiki?curid=49670559,Judicial system in the United Arab Emirates


In [36]:
ds_small = df_big.sample(n=32000, random_state=42122)

In [37]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Load stopwords
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numeric characters
    text = re.sub(r'\d+', '', text)
    # Tokenize
    tokens = text.split()
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Return the preprocessed text as a single string
    return ' '.join(tokens)

In [55]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have a DataFrame called df with columns 'text' and 'label'
# Example:
# df = pd.DataFrame({
#     'text': ['This is text one', 'This is text two', 'Another text', ...],
#     'label': [0, 1, 0, ...]
# })
# Split the data into training and testing sets
ds_small['text'] = ds_small['text'].apply(preprocess_text)
X = ds_small['text']
ds_small['category_encoded'] = ds_small['category'].astype('category').cat.codes
y = ds_small['category_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def build_model(X_train:pd.core.series.Series):

    # Prepare data for Doc2Vec
    # `TaggedDocument` is needed for training the Doc2Vec model.
    tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(X_train)]

    # Train Doc2Vec Model
    doc2vec_model = Doc2Vec(vector_size=512,  # Dimensionality of the feature vectors
                            window=4,       # The maximum distance between the current and predicted word
                            min_count=2,    # Ignores words with a total frequency lower than this
                            workers=8,      # Number of worker threads to use
                            epochs=40)      # Number of training iterations

    doc2vec_model.build_vocab(tagged_data)  # Build vocabulary
    doc2vec_model.train(tagged_data, total_examples=len(tagged_data), epochs=doc2vec_model.epochs)  # Train model
    return doc2vec_model

# Transform the data into vectors
# Use the trained Doc2Vec model to convert the text data into vector representations.
def infer_vectors(doc2vec_model:Doc2Vec, X:pd.core.series.Series):
    vector = [doc2vec_model.infer_vector(text.split()) for text in X]
    return vector


In [ ]:
# Train a classifier
# Here, we use RandomForestClassifier but you can choose any classifier (e.g., SVM, Logistic Regression, etc.)
doc2vec_model = build_model(X_train)
X_train_vectors, X_test_vectors = infer_vectors(doc2vec_model, X_train), infer_vectors(doc2vec_model, X_test)
clf = RandomForestClassifier(n_estimators=300,random_state=42,verbose=1,n_jobs=8)
clf.fit(X_train_vectors, y_train)

# Predict and evaluate the model on the test set
y_pred = clf.predict(X_test_vectors)

# Compute accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    4.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   20.8s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:   34.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.2s finished


Accuracy: 0.2721875
Classification Report:
              precision    recall  f1-score   support

           0       0.36      0.13      0.19       301
           1       0.30      0.12      0.17       336
           2       0.35      0.50      0.42       454
           3       0.44      0.28      0.35       334
           4       0.16      0.04      0.06       209
           5       0.31      0.16      0.21       281
           6       0.24      0.38      0.29       567
           7       0.21      0.18      0.19       447
           8       0.15      0.06      0.08       595
           9       0.39      0.27      0.32       408
          10       0.33      0.54      0.41       554
          11       0.39      0.12      0.18       156
          12       0.00      0.00      0.00        90
          13       0.27      0.02      0.03       220
          14       0.27      0.47      0.35       758
          15       0.16      0.23      0.19       690

    accuracy                         

In [48]:
# Initialize and train the XGBoost model
def train_model(X_train, X_test, y_train, y_test):
    xgb_model = XGBClassifier(use_label_encoder=False, 
                              eval_metric='mlogloss', 
                              objective='multi:softmax', 
                              num_class= 16,
                              learning_rate=0.1, 
                              #max_depth=5, 
                              n_estimators=20,
                              gemma=1
                              )

    # Train the XGBoost model with early stopping on the validation set
    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)
    return xgb_model

from sklearn.metrics import classification_report

def evaluate(_X_test:np.array, _y_test:list, xgb_model):
    _y_pred = xgb_model.predict(_X_test)
    accuracy = accuracy_score(_y_test, _y_pred)
    print(f"Accuracy: {accuracy}")
    print("Classification Report:\n",classification_report(_y_test, _y_pred)) 
    return accuracy

# Extract epoch-wise metrics
def visualize_model(xgb_model):
    from matplotlib import pyplot as plt
    results = xgb_model.evals_result_  # Get the evaluation results during training

    # Print results keys
    print("Keys in evals_result_:", results.keys())
    # Accessing specific metrics
    validation_loss = results['validation_0']['mlogloss']  # Epoch-wise loss on validation set
    epochs = range(1, len(validation_loss) + 1)  # Epoch numbers
    # Plot the loss over epochs
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, validation_loss, label='Validation Loss', marker='o')
    plt.title('Validation Loss Across Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Log Loss')
    plt.legend()
    plt.grid()
    plt.show()

In [49]:
model = train_model(X_train_vectors, X_test_vectors, y_train, y_test)
evaluate(X_test_vectors, y_test, model)

/Users/ngumus/projects/Master-Thesis/.venv/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [00:33:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "gemma", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-mlogloss:2.71583
[1]	validation_0-mlogloss:2.66875
[2]	validation_0-mlogloss:2.62810
[3]	validation_0-mlogloss:2.59179
[4]	validation_0-mlogloss:2.56201
[5]	validation_0-mlogloss:2.53393
[6]	validation_0-mlogloss:2.50653
[7]	validation_0-mlogloss:2.48187
[8]	validation_0-mlogloss:2.45986
[9]	validation_0-mlogloss:2.43875
[10]	validation_0-mlogloss:2.41934
[11]	validation_0-mlogloss:2.40094
[12]	validation_0-mlogloss:2.38423
[13]	validation_0-mlogloss:2.36868
[14]	validation_0-mlogloss:2.35326
[15]	validation_0-mlogloss:2.33820
[16]	validation_0-mlogloss:2.32558
[17]	validation_0-mlogloss:2.31211
[18]	validation_0-mlogloss:2.29966
[19]	validation_0-mlogloss:2.28672
Accuracy: 0.28796875
Classification Report:
               precision    recall  f1-score   support

           0       0.37      0.26      0.31       301
           1       0.30      0.23      0.26       336
           2       0.43      0.46      0.44       454
           3       0.36      0.23      0.28     

0.28796875

In [30]:
y_train

7125     Religion&Philosophy&Ethics
15247        Mechanical_engineering
1675                     Psychology
9919               Natural_sciences
15458             Civil_engineering
                    ...            
10776        Engineering&Technology
14332                       Biology
5318             Business&Economics
5053     Religion&Philosophy&Ethics
9542             Business&Economics
Name: category, Length: 12800, dtype: object